In [1]:
using Revise, StatsBase, DataFrames, CSV, QARBoM, DWave

    CondaPkg Found dependencies: /Users/pripper/.julia/packages/DWave/i51VN/CondaPkg.toml
    CondaPkg Found dependencies: /Users/pripper/.julia/packages/PythonCall/S5MOg/CondaPkg.toml
    CondaPkg Found dependencies: /Users/pripper/.julia/packages/QiskitOpt/ocMRt/CondaPkg.toml
    CondaPkg Dependencies already up to date


In [33]:
MOI = QARBoM.ToQUBO.MOI
learning_rate = 0.0001
# num_sweeps = [i for i in 1:25:101]
# num_reads = [i for i in 1:25:101]
# W = randn(22,10)

0.0001

In [3]:
df = DataFrame(CSV.File("./tep_normal_train.csv"))

Row,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,62.92,53.815,24.55,60.741,22.234,40.215,39.663,49.113,47.46,40.481,18.934,50,0.25147,3671.6,4523.1,9.4618,26.5,41.882,2705.4,74.73,120.4,0.33588,80.175,50.531,2633.9,25.082,51.115,3101.2,22.601,65.76,230.14,341.52,94.598,77.309,32.188,8.8933,26.383,6.882,18.776,1.6567,32.958,13.823,23.978,1.2565,18.579,2.2633,4.8436,2.2986,0.017866,0.8357,0.098577,53.724,43.828
2,63.24,54.252,24.792,59.78,22.189,40.207,39.064,46.322,47.421,41.409,19.877,50,0.24901,3653.3,4486.6,9.3912,26.955,41.995,2705.2,75.237,120.38,0.33595,80.139,50.328,2634.2,26.452,49.909,3102.3,22.201,65.724,230.58,341.47,94.554,77.389,32.188,8.8933,26.383,6.882,18.776,1.6567,32.958,13.823,23.978,1.2565,18.579,2.2633,4.8436,2.2986,0.017866,0.8357,0.098577,53.724,43.828
3,63.61,53.772,24.62,60.108,22.001,40.346,38.205,49.821,47.371,40.921,17.361,50,0.25084,3631.5,4526.6,9.3101,26.496,42.267,2704.5,74.697,120.39,0.3348,80.081,50.036,2632.7,23.796,51.42,3101.4,23.269,65.735,231.12,341.24,94.62,77.206,32.159,8.8967,26.721,6.7772,18.83,1.6772,33.089,13.698,24.442,1.2743,18.54,2.2211,4.7767,2.4316,0.017866,0.8357,0.098577,53.724,43.828
4,63.5,53.881,24.78,62.752,22.101,39.936,43.227,47.4,47.466,41.534,17.809,50,0.24922,3638.2,4517.5,9.3723,26.818,42.661,2702.3,74.875,120.41,0.33816,79.911,51.742,2630.7,26.252,50.374,3100.5,23.091,65.723,230.07,340.95,94.615,77.029,32.159,8.8967,26.721,6.7772,18.83,1.6772,33.089,13.698,24.442,1.2743,18.54,2.2211,4.7767,2.4316,0.017866,0.8357,0.098577,53.724,43.828
5,63.44,54.328,24.97,58.28,22.294,38.133,36.967,45.92,47.479,40.748,19.651,50,0.25309,3698.0,4468.7,9.5389,26.501,42.161,2704.7,75.419,120.39,0.32481,80.117,49.615,2633.9,26.014,49.735,3102.8,22.315,65.696,229.96,341.51,94.516,76.968,32.272,8.8253,26.102,6.9838,18.557,1.647,32.984,13.876,24.189,1.2346,18.82,2.2859,4.8002,2.3512,0.017866,0.8357,0.098577,53.724,43.828
6,63.38,53.878,24.933,61.113,21.974,38.619,38.901,47.832,47.448,40.962,19.102,50,0.25346,3701.1,4506.3,9.3432,27.117,42.453,2704.9,75.545,120.42,0.32083,80.055,50.272,2634.0,25.764,50.561,3102.8,22.531,65.707,230.3,341.18,94.548,77.477,32.272,8.8253,26.102,6.9838,18.557,1.647,32.984,13.876,24.189,1.2346,18.82,2.2859,4.8002,2.3512,0.0065433,0.82241,0.10181,53.281,43.464
7,62.46,54.336,23.749,59.756,22.036,40.935,39.451,47.054,47.481,41.047,17.135,50,0.2432,3649.3,4525.3,9.4133,26.635,42.441,2703.8,73.318,120.41,0.34048,80.135,50.459,2632.8,25.203,50.225,3101.6,23.362,65.724,229.95,341.35,94.569,77.342,32.623,9.0641,26.402,6.9635,18.526,1.6308,32.632,13.812,23.826,1.2491,18.163,2.2721,4.8852,2.2702,0.0065433,0.82241,0.10181,53.281,43.464
8,62.19,54.243,23.752,60.048,22.216,40.613,37.995,48.332,47.346,41.409,16.929,50,0.24317,3665.5,4533.0,9.3,26.976,42.071,2705.0,74.792,120.4,0.34311,80.149,49.964,2634.1,26.06,50.777,3101.7,23.453,65.754,231.45,341.33,94.614,77.182,32.623,9.0641,26.402,6.9635,18.526,1.6308,32.632,13.812,23.826,1.2491,18.163,2.2721,4.8852,2.2702,0.0065433,0.82241,0.10181,53.281,43.464
9,62.73,53.912,20.341,60.957,22.067,39.928,42.971,46.869,47.488,40.589,17.117,50,0.20453,3639.3,4570.4,9.2843,27.0,42.325,2704.6,75.18,120.38,0.33499,80.127,51.655,2633.3,23.956,50.145,3101.7,23.373,65.75,229.85,341.43,94.655,77.237,32.494,8.8458,26.37,6.9191,18.506,1.6501,32.812,13.684,23.857,1.3654,18.717,2.3004,4.8546,2.2831,0.0065433,0.82241,0.10181,53.281,43.464


In [4]:
df_zscore = select(df,All() .=> zscore) 
select!(df_zscore, Not(:C12_zscore))
df_zscore

Row,C1_zscore,C2_zscore,C3_zscore,C4_zscore,C5_zscore,C6_zscore,C7_zscore,C8_zscore,C9_zscore,C10_zscore,C11_zscore,C13_zscore,C14_zscore,C15_zscore,C16_zscore,C17_zscore,C18_zscore,C19_zscore,C20_zscore,C21_zscore,C22_zscore,C23_zscore,C24_zscore,C25_zscore,C26_zscore,C27_zscore,C28_zscore,C29_zscore,C30_zscore,C31_zscore,C32_zscore,C33_zscore,C34_zscore,C35_zscore,C36_zscore,C37_zscore,C38_zscore,C39_zscore,C40_zscore,C41_zscore,C42_zscore,C43_zscore,C44_zscore,C45_zscore,C46_zscore,C47_zscore,C48_zscore,C49_zscore,C50_zscore,C51_zscore,C52_zscore,C53_zscore
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.21751,-0.346618,-0.0296897,-0.448699,0.0212264,0.0632443,0.562581,1.08573,-0.268895,-1.12778,0.535139,0.0290887,0.273594,0.307986,1.21377,-1.95851,-2.10108,-0.00595511,-0.519519,0.00580843,-0.117296,0.302436,0.562464,-0.02774,-0.115658,1.08634,-0.216348,-0.545647,-0.18869,-0.281922,0.0555223,-0.0229507,0.0821221,0.00510033,0.0338113,-0.0726283,-0.00271398,0.00207276,0.0523565,0.0210443,-0.00468791,-0.0487744,0.046596,-0.0264244,0.0449685,0.0876585,-0.00565699,-0.0717666,0.0090431,0.0266694,-0.0448126,-0.0816862
2,0.342977,0.522908,0.0427723,-1.21306,-0.0414842,0.058377,0.361824,-0.116792,-0.280796,0.560124,1.15076,-0.0431716,-0.274388,-0.53662,0.469163,0.231979,-1.59054,-0.0287135,0.371002,-1.06652,-0.112129,0.169711,0.362243,0.00449499,1.22289,-0.116194,-0.0572064,-1.16547,-0.265458,-0.246995,0.0343387,-0.334147,0.381303,0.00510033,0.0338113,-0.0726283,-0.00271398,0.00207276,0.0523565,0.0210443,-0.00468791,-0.0487744,0.046596,-0.0264244,0.0449685,0.0876585,-0.00565699,-0.0717666,0.0090431,0.0266694,-0.0448126,-0.0816862
3,0.991039,-0.432178,-0.00872964,-0.952177,-0.303475,0.142945,0.073928,1.39077,-0.296053,-0.32748,-0.491769,0.010583,-0.927175,0.388976,-0.386184,-1.97777,-0.361653,-0.108368,-0.577482,-0.530355,-0.197007,-0.0441251,0.074239,-0.15668,-1.37213,1.39046,-0.187413,0.48945,-0.242001,-0.20413,-0.063106,0.132647,-0.303074,-0.0860144,0.0678041,0.979971,-0.999531,0.164681,0.837896,0.368177,-1.07336,1.15395,0.217374,-0.121479,-1.57099,-0.871981,2.44776,-0.0717666,0.0090431,0.0266694,-0.0448126,-0.0816862
4,0.798372,-0.215294,0.0391792,1.15082,-0.164118,-0.106501,1.75706,0.347671,-0.267064,0.787482,-0.199299,-0.0370031,-0.726548,0.178403,0.269828,-0.427574,1.41843,-0.35871,-0.264833,0.541971,0.050984,-0.670885,1.75689,-0.37158,1.02748,0.347469,-0.317619,0.213631,-0.26759,-0.287479,-0.185971,0.0972841,-0.965012,-0.0860144,0.0678041,0.979971,-0.999531,0.164681,0.837896,0.368177,-1.07336,1.15395,0.217374,-0.121479,-1.57099,-0.871981,2.44776,-0.0717666,0.0090431,0.0266694,-0.0448126,-0.0816862
5,0.693281,0.67413,0.0960709,-2.40614,0.10484,-1.20346,-0.340991,-0.289996,-0.263097,-0.642143,1.00322,0.0766748,1.06413,-0.950824,2.02693,-1.95369,-0.840559,-0.0856094,0.690677,-0.530355,-0.934339,0.0886005,-0.340999,-0.02774,0.794944,-0.289694,0.0151305,-0.988817,-0.325166,-0.29621,0.0512856,-0.602907,-1.19314,0.269019,-0.646046,-0.947718,0.965568,-0.657395,-0.319338,0.0899408,0.448429,0.498156,-0.163519,0.560964,0.910386,-0.534888,0.96464,-0.0717666,0.0090431,0.0266694,-0.0448126,-0.0816862
6,0.588189,-0.221263,0.084992,-0.152816,-0.341101,-0.907771,0.307194,0.533801,-0.272557,-0.252906,0.644815,0.0875432,1.15695,-0.080764,-0.0370844,1.01189,0.478691,-0.062851,0.91199,1.07813,-1.22809,-0.139983,0.307009,-0.016995,0.550684,0.533932,0.0151305,-0.654115,-0.301709,-0.269221,-0.0885264,-0.376582,0.710402,0.269019,-0.646046,-0.947718,0.965568,-0.657395,-0.319338,0.0899408,0.448429,0.498156,-0.163519,0.560964,0.910386,-0.534888,0.96464,-1.22995,-0.553215,0.341666,-0.923

In [5]:
x_train = Vector{Vector{Float64}}()

for row in eachrow(df_zscore)
    push!(x_train, collect(row))
end

In [6]:
max_vis = Vector{Float64}()
min_vis = Vector{Float64}()
for col in names(df_zscore)
    push!(max_vis, maximum(df_zscore[!, col]))
    push!(min_vis, minimum(df_zscore[!, col]))
end

In [25]:
W = randn(52,20)

52×20 Matrix{Float64}:
 -0.630998   0.0627119   1.32649    …  -1.22711   -0.194638    0.904101
 -1.25972   -0.290617   -0.835813       0.421357  -1.40743    -0.649197
  0.701632  -1.55548     1.88513        0.366241   1.00583    -0.0404718
 -1.08816   -1.0922     -1.22798        0.692537  -1.46276     0.286778
  0.527432  -0.351304   -0.430594       0.907143  -0.709788    0.525558
  1.32866   -1.03801     1.93199    …   0.784208  -2.21983    -1.22068
 -1.46744   -1.7046      0.771538      -0.62002    1.03885     1.05052
  0.332231  -0.234479    0.567584       1.15901    1.3404      1.07771
  0.331828  -0.400661   -1.64281       -1.20338    0.122426    0.91394
 -1.31791    0.1241     -0.0197188     -0.746697   0.818181   -0.747578
  0.265019   0.294473   -1.23485    …  -1.71286    1.10795    -0.0163776
 -0.608679  -1.99097     0.989798      -1.63853    0.113446    0.428562
 -0.290484  -0.738952    0.128721      -0.48029    1.13823    -0.141453
  ⋮                                 ⋱      

In [35]:
rbm_dwave = GRBM(52,20, W, max_vis, min_vis)
rbm_pcd = RBM(52,20, W)

RBM([-0.6309977612487733 0.06271185525491246 … -0.1946378644984662 0.9041014357991419; -1.2597190990582277 -0.29061738939782517 … -1.407433555042847 -0.6491971149065235; … ; 1.3372810763147984 -0.6132926899950064 … -0.987833994533511 -1.4519974242691167; 0.15461489008808477 1.4528018636879474 … 0.8898976225091593 -1.741019987350605], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 52, 20)

In [36]:
MOI.supports(::DWave.Neal.Optimizer, ::MOI.ObjectiveSense) = true
function setup_dwave(model, sampler)
    MOI.set(model, MOI.RawOptimizerAttribute("num_reads"), 25)
    MOI.set(model, MOI.RawOptimizerAttribute("num_sweeps"), 100)
end

setup_dwave (generic function with 1 method)

In [37]:
mse_pcd = QARBoM.train_pcd!(rbm_pcd, x_train; n_epochs = 50, batch_size = 10, learning_rate = learning_rate)

Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Gibbs) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |   60.6532 |        0.0055 |       0.0178 |        0.0172 |    0.0405 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Gibbs) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |   60.1736 |        0.0050 |       0.0148 |        0.0152 |    0.0755 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Gibbs) | Time (Update) | T

50-element Vector{Float64}:
 60.65324234745741
 60.17362124651761
 59.70943189469933
 59.25699304767659
 58.82091286404062
 58.39993094722766
 57.99431858123548
 57.60522488560673
 57.230450102796034
 56.87224434667882
 56.527935294796684
 56.198957889493215
 55.87953083780849
  ⋮
 50.47802579506254
 50.345960432601885
 50.21881183460216
 50.095866566968944
 49.97600286551008
 49.859829379151876
 49.74875573702861
 49.64156004215772
 49.53511538650661
 49.430122151650416
 49.328608351269125
 49.23105868078933

In [38]:
mse_dwave = QARBoM.train_persistent_qubo!(
    rbm_dwave, 
    x_train;
    batch_size = 10, 
    n_epochs = 50,  
    learning_rate = [learning_rate for i in 1:50],
    model_setup = setup_dwave,
    sampler = DWave.Neal.Optimizer
)

Setting up QUBO model
Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |   60.3358 |        0.0048 |       6.6302 |        0.1286 |    6.7635 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |   59.3808 |        0.0047 |       6.2552 |        0.1509 |   13.1743 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsam

50-element Vector{Float64}:
 60.335755544157124
 59.38078021831938
 58.58143798525895
 57.93395548686425
 57.40894822025034
 56.97235743142489
 56.605514710900124
 56.29578290875637
 56.03045660512465
 55.800435882584765
 55.60184745177148
 55.43871598373892
 55.29939376824109
  ⋮
 54.02074912814136
 54.008370575496116
 53.99681478645512
 53.982491382676486
 53.968174664764426
 53.953874298418256
 53.92961139177529
 53.90387653190797
 53.88834900384644
 53.87516205331946
 53.86048052939223
 53.842480431397625

In [41]:
# df = DataFrame(
#     "mse_dwave" => mse_dwave,
#     "mse_pcd" => mse_pcd)
# CSV.write("mse_gaussian_first.csv", df)

"mse_gaussian_first.csv"

In [42]:
df_loaded = DataFrame(CSV.File("mse_gaussian_first.csv"))

Row,mse_dwave,mse_pcd
,Float64,Float64
1,60.3358,60.6532
2,59.3808,60.1736
3,58.5814,59.7094
4,57.934,59.257
5,57.4089,58.8209
6,56.9724,58.3999
7,56.6055,57.9943
8,56.2958,57.6052
9,56.0305,57.2305


In [39]:
using Plots

In [45]:
plot(Matrix(df_loaded), labels=permutedims(names(df_loaded)), linewidth=2)
theme(:wong)
# lens!([49, 50], [1.88, 2.5], inset = (1, bbox(0.5, 0.0, 0.4, 0.4)), subplot=2)

plot!(legend=:outerbottom, legendcolumns=3)

xlabel!("Epoch", subplot =1)
ylabel!("Mean Squared Error", subplot =1 )

savefig("tep_normal.pdf")

┌ Warning: n° of legend_column=3 is larger than n° of series=2
└ @ Plots ~/.julia/packages/Plots/du2dt/src/backends/gr.jl:1235


"/Users/pripper/Documents/GitHub/RBM/example/TEP/tep_normal.pdf"

In [44]:
using QARBoM.JuMP

In [13]:
model = QARBoM._create_qubo_model(rbm_dwave, DWave.Neal.Optimizer, setup_dwave)

A JuMP Model
Minimization problem with:
Variables: 72
Objective function type: JuMP.QuadExpr
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 52 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 52 constraints
`JuMP.VariableRef`-in-`MathOptInterface.ZeroOne`: 20 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Virtual QUBO Model
Names registered in the model: hid, vis

In [19]:
QARBoM._qubo_sample(rbm_dwave, model)

total_samples = 9
(v_sampled, h_sampled) = ([34.29886803259219, -6.175837401680726, -27.016496272106007, -31.097230656664973, -21.795292433708774, -16.62921285937509, -4.041532164538292, -8.032723158068746, -17.09390703588011, -26.765295157111762, -27.863306842400174, -26.809387690754964, 27.60662989590069, 15.334523868140414, -26.873783292044315, -28.588641367120644, -33.14132835589356, 1.619144373159635, 29.801730125046998, 29.00507710375434, 23.93391007504793, -10.605941921112304, -4.040509429215036, 28.375039377531515, 22.506745462746874, 27.500955706580708, -5.636311182228074, 27.415819272286985, 21.43870736160903, 12.34397026839731, 25.6658456394348, -16.494831958310925, -27.836438708241303, -0.05149552541370106, -25.322334523630623, -26.439227655519502, 5.657812631830353, -13.640467656916513, 29.923184192485493, -14.861835852990605, 7.2675326148281805, 28.861979528286085, -14.900196911831499, 20.64493683141899, -29.854254857516494, -29.34287966016489, -21.86600267462579, 7.32613

([3.8109853369546878, -0.6862041557423029, -3.0018329191228896, -3.4552478507405526, -2.421699159300975, -1.8476903177083435, -0.4490591293931435, -0.8925247953409718, -1.899323003986679, -2.973921684123529  …  -1.6555774346479444, 2.2938818701576653, -3.317139428612944, -3.260319962240543, -2.4295558527361987, 0.814014991751182, 1.9854327794258255, 3.2626629382862475, 2.6444367683537524, -2.601148167635623], [1.0, 1.0, 0.1111111111111111, 1.0, 0.5555555555555556, 0.6666666666666666, 0.8888888888888888, 0.5555555555555556, 1.0, 1.0, 0.6666666666666666, 0.4444444444444444, 0.7777777777777778, 0.6666666666666666, 1.0, 0.0, 1.0, 0.5555555555555556, 1.0, 0.4444444444444444])

In [17]:
QARBoM.JuMP.value.(model[:vis], result = 1)

52-element Vector{Float64}:
  3.8109853369546887
  3.197147500445586
 -3.001832919122889
 -3.455247850740553
 -2.4216991593009753
 -3.177124296689632
 -3.190458911908922
  3.056034296097594
 -1.899323003986679
 -2.9739216841235296
 -3.0959229824889087
 -2.9788208545283292
  3.0674033217667427
  ⋮
  2.6883682345316045
  3.2068866142540084
 -3.173434235141567
  2.293881870157665
 -3.3171394286129443
 -3.2603199622405428
 -3.0825697005813684
 -2.4597946346606254
  2.5876477975760404
  3.2626629382862475
  2.644436768353752
 -2.601148167635623